In [45]:
import torch as th
import numpy as np
from tqdm.notebook import trange

In [46]:
device = "mps"
dtype = th.float32

In [47]:
x = th.linspace(
    -np.pi,
    np.pi,
    2000,
    device=device,
    dtype=dtype
)
y = th.sin(x)

In [48]:
class Polynomial3(th.nn.Module):
    def __init__(self):
        super(Polynomial3, self).__init__()
        self.a = th.nn.Parameter(th.randn(()))
        self.b = th.nn.Parameter(th.randn(()))
        self.c = th.nn.Parameter(th.randn(()))
        self.d = th.nn.Parameter(th.randn(()))

    def forward(self, x):
        return self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3

In [49]:
model = Polynomial3().to(device=device, dtype=dtype)

# Manual optimization
# Use simple gradient descent to fit the model to our data
steps = 2000
eps = 1e-6
lr = 1e-6
logging_steps = 100
for t in trange(steps):
    y_pred = model(x)
    loss = th.square(y_pred - y).sum()
    model.zero_grad()
    if t % logging_steps == logging_steps - 1:
        print(t, loss.item())
    loss.backward()
    with th.no_grad():
        for param in model.parameters():
            param -= lr * param.grad

  0%|          | 0/2000 [00:00<?, ?it/s]

0 143060.140625
100 507.24456787109375
200 346.46881103515625
300 237.79769897460938
400 164.27020263671875
500 114.46917724609375
600 80.7024154663086
700 57.78276824951172
800 42.20869827270508
900 31.614208221435547
1000 24.399005889892578
1100 19.47964859008789
1200 16.121774673461914
1300 13.827129364013672
1400 12.257232666015625
1500 11.181941986083984
1600 10.444584846496582
1700 9.938377380371094
1800 9.590476036071777
1900 9.351079940795898


In [50]:
import pandas as pd
import plotly.express as px

In [51]:
x = x
y = y
y_pred = model(x)

data = pd.DataFrame({
    "x": x.cpu().numpy(),
    "y": y.cpu().numpy(),
    "y_pred": y_pred.detach().cpu().numpy()
})

In [52]:
fig = px.line(data, x="x", y=["y", "y_pred"], title="Fitting a polynomial to data")
fig.show()